# Calculating the Mulliken population

In [Cedillo2012](https://link.springer.com/article/10.1007/s00214-012-1227-6), we find a Mulliken charge of 0.20 on the carbon atom in CO. In this notebook, we will reproduce this result.

In [1]:
# Force the local gqcpy to be imported
import sys
sys.path.insert(0, '../../build/gqcpy/')

import gqcpy
import numpy as np

Let's start by creating the molecule, and quantizing the Hamiltonian in the associated restricted spin-orbital basis of the AOs.

In [2]:
molecule = gqcpy.Molecule.ReadXYZ("../../gqcp/tests/data/CO_mulliken.xyz" , 0)  # '0': Create a neutral molecule.
N = molecule.numberOfElectrons()

spin_orbital_basis = gqcpy.RSpinOrbitalBasis(molecule, "STO-3G")
S = spin_orbital_basis.quantizeOverlapOperator()

hamiltonian = gqcpy.RSQHamiltonian.Molecular(spin_orbital_basis, molecule)

Afterwards, we should solve the RHF SCF equations.

In [3]:
environment = gqcpy.RHFSCFEnvironment.WithCoreGuess(N, hamiltonian, S.parameters())
solver = gqcpy.RHFSCFSolver.DIIS()
objective = gqcpy.DiagonalRHFFockMatrixObjective(hamiltonian)  # This objective makes sure that the RHF parameters represent the canonical RHF MOs.

rhf_parameters = gqcpy.RHF.optimize(objective, solver, environment).groundStateParameters()
D = rhf_parameters.calculateScalarBasis1DM()  # The AO density matrix.

The Mulliken population is effectively the expectation value of the Mulliken operator and a density matrix, so we'll have to set up the Mulliken operator.

Since we're interested in determining the Mulliken charge on the carbon atom, we'll prepare a Mulliken partitioning of the spin-orbital basis with basis functions centered on the carbon atom.

In [4]:
mulliken_partitioning = spin_orbital_basis.mullikenPartitioning(lambda shell: shell.nucleus().element() == "C")

Let's also confirm that the supplied selector lambda function has selected the correct indices for the basis functions centered on the carbon atom.

In [5]:
print(mulliken_partitioning.indices())

[0, 1, 2, 3, 4]


Theoretically, the Mulliken operator is the Mulliken-partitioned number/overlap operator. Since GQCP supports partitioning of any one-electron operator, we can use the `.partitioned()` API.

In [6]:
mulliken_operator = S.partitioned(mulliken_partitioning)

What remains is to calculate the expectation value of the Mulliken-partitioned number operator, which yields the Mulliken population. The Mulliken charge can then be calculated straightforwardly.

In [7]:
C_population = mulliken_operator.calculateExpectationValue(D)[0]
print("Mulliken population: ", C_population)

Mulliken population:  5.801066370434233


In [8]:
C_charge = 6 - C_population
print("Mulliken charge: ", C_charge)

Mulliken charge:  0.19893362956576688
